In [7]:
#import libraries
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
#from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
#import subprocess
import pandas as pd
import pickle
import re

In [105]:
sport = 'Football'
live = True
betX = {'3-Way': 1, 'Over/Under': 2, 'Both teams to score?': 10}

In [107]:
for i in betX: 
    print(betX[i])

1
2
10


In [108]:
#changing chromedriver default options
options = Options()
options.headless = False
options.add_argument('window-size=1920x1080') 

if live:
    web = 'https://sports.bwin.com/en/sports/live/football-4?fallback=false'
else:
    web = 'https://sports.bwin.com/en/sports/football-4/betting'
    
#execute chromedriver with edited options
driver = webdriver.Chrome('./chromedriver', options=options)
driver.get(web)
#driver.maximize_window()

In [109]:
# try to close ad
try:
    accept = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="messages-with-overlay"]/div/vn-content-message/div/span')))
    accept.click()
except:
    pass

In [110]:
#Initialize your storage
teams = []
odds_events = []
#select values from dropdown
bet_odds = []

In [86]:
# weitere mögliche Dropdowns:
# - Total 1st Half Over/Under
# - Total 2nd Half Over/Under

In [119]:
# if headless = False -> kann nur 2 Dropdowns machen

for i, bet_name in enumerate(betX):
    bet_odds.append([]) # initialize list analogously to x12=[], over_under = [], bttw = []
    xpath = '//*[@id="main-view"]/ms-live/ms-live-event-list/div/ms-grid/ms-grid-header/div/ms-group-selector['+str(i+1)+']/ms-dropdown/div/span[2]'
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, xpath))).click() 
    bet_num = str(betX[bet_name])
    xpath = '//*[@id="main-view"]/ms-live/ms-live-event-list/div/ms-grid/ms-grid-header/div/ms-group-selector['+str(i+1)+']/ms-dropdown/div[2]/div[' + bet_num + ']'
    #xpath = '//*[@id="main-view"]/ms-live/ms-live-event-list/div/ms-grid/ms-grid-header/div/ms-group-selector['+str(i+1)+']/ms-dropdown/div[2]/div[1]'
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpath))).click()

In [101]:
box = driver.find_element_by_xpath('//ms-grid[contains(@sortingtracking,"Live")]') #livebox
rows = WebDriverWait(box, 5).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'grid-event')))
len(rows)

In [ ]:


      
    #Looking for live events 'Program_LIVE'
    box = driver.find_element_by_xpath('//div[contains(@testid, "Program_LIVE")]')
    #Looking for 'sports titles'
    sport_title = box.find_elements_by_class_name('SportTitle-styles-sport')
    
    for sport in sport_title:
        # selecting only football
        if sport.text == sport:
            parent = sport.find_element_by_xpath('./..') #immediate parent node
            grandparent = parent.find_element_by_xpath('./..') #grandparent node = the whole 'football' container
        
            #1 #empty groups
            try:
                empty_groups = grandparent.find_elements_by_class_name('EventOddGroup-styles-empty-group')
                empty_events = [empty_group.find_element_by_xpath('./..') for empty_group in empty_groups]####
            except:
                pass
        
            #Single row event
            single_row_events = grandparent.find_elements_by_class_name('EventRow-styles-event-row')
            #2 #remove empty events from single_row_events
            try:
                #empty_events
                single_row_events = [single_row_event for single_row_event in single_row_events if single_row_event not in empty_events]
            except:
                pass
        
            #3 #Getting Data
            for match in single_row_events:
                odds_event = match.find_elements_by_class_name('EventOddGroup-styles-odd-groups')
                odds_events.append(odds_event)
                # teams
                for team in match.find_elements_by_class_name('EventTeams-styles-titles'):
                    teams.append(team.text)
            for odds_event in odds_events:
                for n, box in enumerate(odds_event):
                    rows = box.find_elements_by_xpath('.//*')
                    if n == 0:
                        x12.append(rows[0].text)
                    # over/under + goal line
                    if n == 1:
                        parent = box.find_element_by_xpath('./..')
                        goals = parent.find_element_by_class_name('EventOddGroup-styles-fixed-param-text').text
                        over_under.append(goals+'\n'+rows[0].text)
                    # both teams to score
                    if n == 2:
                        btts.append(rows[0].text)

    driver.quit()
    
    # make Readable
    pd.set_option('display.max_rows', 500)
    pd.set_option('display.max_columns', 500)
    pd.set_option('display.width', 1000)

    #Storing lists within dictionary
    dict_gambling = dict(zip(betX, bet_odds)) 
    dict_gambling['Teams'] = teams
    #Presenting data in dataframe
    df_bookie = pd.DataFrame.from_dict(dict_gambling)
    #clean leading and trailing whitespaces
    df_bookie = df_tipico.applymap(lambda x: x.strip() if isinstance(x, str) else x) #14.0\n or 14 or \n14.0
    
    #Save data with Pickle
    output = open('ScrappedData/df_bwin', 'wb') #don't forget to change name_of_file
    pickle.dump(df_bookie, output)
    output.close()
    
    return df_bookie